In [ ]:
from lxml import etree
from lxml.etree import tostring

tree = etree.parse('root.xml')
root = tree.getroot()
ID_list = []
text_list = []
for page in root.findall(".//page"):
    # print(tostring(title))
    print(page.text)
    print(len(page.getchildren()))

    items = page.getchildren()

    id_value = ""
    text_value = ""

    for id in items:
            if "id" in str(id.tag):
                # print("ID: "+str(id.text)+ " - "+str(page.find(".//title", namespaces=root.nsmap).text))
                id_value = str(id.text)
                ID_list.append(id_value)
                id.clear()
    for revision in page.findall(".//revision", namespaces=root.nsmap):
            revisioni = revision.getchildren()
            revision.clear()

    for text in revisioni:
            if "text" in str(text.tag):
                # print(text.text)
                text_list.append(text.text.replace("\n", ""))
    page.clear()


In [ ]:
import csv

with open('file.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ID", "TEXT"])
    for i in range(len(ID_list)):
        writer.writerow([ID_list[i], text_list[i]])